### Convolution Operation Function

In [1]:
def conv(input, kernel, biases, k_h, k_w, c_o, s_h, s_w,  padding="VALID", group=1):
    '''
    From https://github.com/ethereon/caffe-tensorflow
    '''
    print("\n inside conv printing input :", input.shape)
    print(" inside conv printing biases :", biases.shape)
    print(" inside conv printing k_h, k_w, c_o, s_h, s_w :", k_h, k_w, c_o, s_h, s_w)

    c_i = input.get_shape()[-1]
    assert c_i % group == 0
    assert c_o % group == 0
    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding)
    

    if tf.__version__ < "1.0.0":
        if group == 1:
            conv = convolve(input, kernel)
        else:
            input_groups = tf.split(3, group, input)
            kernel_groups = tf.split(3, group, kernel)
            output_groups = [convolve(i, k) for i, k in zip(input_groups, kernel_groups)]
            conv = tf.concat(3, output_groups)
    else:
        if group == 1:
            conv = convolve(input, kernel)
        else:
            input_groups = tf.split(input, group, 3)
            print(" inside conv printing input_groups :", input_groups)  
            kernel_groups = tf.split(kernel, group, 3)
            print(" inside conv printing kernel_groups :", kernel_groups)  
            output_groups = [convolve(i, k) for i, k in zip(input_groups, kernel_groups)]
            print(" inside conv printing output_groups :", output_groups)  
            conv = tf.concat(output_groups, 3)
    print(" inside conv printing conv :", conv)  
    conv_return = tf.reshape(tf.nn.bias_add(conv, biases), [-1] + conv.get_shape().as_list()[1:])
    print(" inside conv printing conv_return :", conv_return)
    return conv_return


### AlexNet Function

In [2]:
def AlexNet(features, feature_extract=False):
    """
    Builds an AlexNet model, loads pretrained weights
    """
    net_data = np.load("bvlc-alexnet.npy", allow_pickle=True, encoding="latin1").item()
    # conv1
    # conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
    k_h = 11
    k_w = 11
    c_o = 96
    s_h = 4
    s_w = 4
    conv1W = tf.Variable(net_data["conv1"][0])
    conv1b = tf.Variable(net_data["conv1"][1])
    conv1_in = conv(features, conv1W, conv1b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=1)
    conv1 = tf.nn.relu(conv1_in)
    print("\n inside AlexNet printing conv1 :", conv1)

    # lrn1
    # lrn(2, 2e-05, 0.75, name='norm1')
    radius = 2
    alpha = 2e-05
    beta = 0.75
    bias = 1.0
    lrn1 = tf.nn.local_response_normalization(conv1, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)
    print(" inside AlexNet printing lrn1 :", lrn1)

    # maxpool1
    # max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
    k_h = 3
    k_w = 3
    s_h = 2
    s_w = 2
    padding = 'VALID'
    maxpool1 = tf.nn.max_pool(lrn1, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)
    print(" inside AlexNet printing maxpool1 :", maxpool1)

    # conv2
    # conv(5, 5, 256, 1, 1, group=2, name='conv2')
    k_h = 5
    k_w = 5
    c_o = 256
    s_h = 1
    s_w = 1
    group = 2
    conv2W = tf.Variable(net_data["conv2"][0])
    conv2b = tf.Variable(net_data["conv2"][1])
    conv2_in = conv(maxpool1, conv2W, conv2b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv2 = tf.nn.relu(conv2_in)
    print("\n inside AlexNet printing conv2 :", conv2)

    # lrn2
    # lrn(2, 2e-05, 0.75, name='norm2')
    radius = 2
    alpha = 2e-05
    beta = 0.75
    bias = 1.0
    lrn2 = tf.nn.local_response_normalization(conv2, depth_radius=radius, alpha=alpha, beta=beta, bias=bias)

    # maxpool2
    # max_pool(3, 3, 2, 2, padding='VALID', name='pool2')
    k_h = 3
    k_w = 3
    s_h = 2
    s_w = 2
    padding = 'VALID'
    maxpool2 = tf.nn.max_pool(lrn2, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)
    print(" inside AlexNet printing maxpool2 :", maxpool2)

    # conv3
    # conv(3, 3, 384, 1, 1, name='conv3')
    k_h = 3
    k_w = 3
    c_o = 384
    s_h = 1
    s_w = 1
    group = 1
    conv3W = tf.Variable(net_data["conv3"][0])
    conv3b = tf.Variable(net_data["conv3"][1])
    conv3_in = conv(maxpool2, conv3W, conv3b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv3 = tf.nn.relu(conv3_in)
    print("\n inside AlexNet printing conv3 :", conv3)


    # conv4
    # conv(3, 3, 384, 1, 1, group=2, name='conv4')
    k_h = 3
    k_w = 3
    c_o = 384
    s_h = 1
    s_w = 1
    group = 2
    conv4W = tf.Variable(net_data["conv4"][0])
    conv4b = tf.Variable(net_data["conv4"][1])
    conv4_in = conv(conv3, conv4W, conv4b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv4 = tf.nn.relu(conv4_in)
    print("\n inside AlexNet printing conv4 :", conv4)

    # conv5
    # conv(3, 3, 256, 1, 1, group=2, name='conv5')
    k_h = 3
    k_w = 3
    c_o = 256
    s_h = 1
    s_w = 1
    group = 2
    conv5W = tf.Variable(net_data["conv5"][0])
    conv5b = tf.Variable(net_data["conv5"][1])
    conv5_in = conv(conv4, conv5W, conv5b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
    conv5 = tf.nn.relu(conv5_in)
    print("\n inside AlexNet printing conv5 :", conv5)


    # maxpool5
    # max_pool(3, 3, 2, 2, padding='VALID', name='pool5')
    k_h = 3
    k_w = 3
    s_h = 2
    s_w = 2
    padding = 'VALID'
    maxpool5 = tf.nn.max_pool(conv5, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)
    print(" inside AlexNet printing maxpool5 :", maxpool5)


    # fc6, 4096
    fc6W = tf.Variable(net_data["fc6"][0])
    fc6b = tf.Variable(net_data["fc6"][1])
    flat5 = tf.reshape(maxpool5, [-1, int(np.prod(maxpool5.get_shape()[1:]))]) 
    fc6 = tf.nn.relu(tf.matmul(flat5, fc6W) + fc6b)
    print("\n inside AlexNet printing fc6 :", fc6)


    # fc7, 4096
    fc7W = tf.Variable(net_data["fc7"][0])
    fc7b = tf.Variable(net_data["fc7"][1])
    fc7 = tf.nn.relu(tf.matmul(fc6, fc7W) + fc7b)
    print(" inside AlexNet printing fc7 :", fc7)

    if feature_extract:
        return fc7

    # fc8, 1000
    fc8W = tf.Variable(net_data["fc8"][0])
    fc8b = tf.Variable(net_data["fc8"][1])

    
    logits = tf.matmul(fc7, fc8W) + fc8b
    probabilities = tf.nn.softmax(logits)
    print("\n inside AlexNet printing fc8 logits :", logits)
    print(" inside AlexNet printing fc8 probabilities :", probabilities)

    return probabilities


In [3]:
import time
#import tensorflow as tf
from caffe_classes import class_names
import numpy as np
from matplotlib.pyplot import imread

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()



# placeholders
x = tf.placeholder(tf.float32, (None, 227, 227, 3))

print("\n inside main printing x:", x)

# By keeping `feature_extract` set to `False`
# we indicate to keep the 1000 class final layer
# originally used to train on ImageNet.
probs = AlexNet(x, feature_extract=False)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# Read Images
im1 = (imread("poodle.png")[:, :, :3] * 255).astype(np.float32)
im1 = im1 - np.mean(im1)

im2 = (imread("weasel.png")[:, :, :3] * 255).astype(np.float32)
im2 = im2 - np.mean(im2)

print("\n inside main printing im1 and im2 shape:", im1.shape, im2.shape)


# Run Inference
t = time.time()
output = sess.run(probs, feed_dict={x: [im1, im2]})


# Print Output
for input_im_ind in range(output.shape[0]):
    inds = np.argsort(output)[input_im_ind, :]
    print("Image", input_im_ind)
    for i in range(5):
        print("%s: %.3f" % (class_names[inds[-1 - i]], output[input_im_ind, inds[-1 - i]]))
    print()

print("Time: %.3f seconds" % (time.time() - t))


Instructions for updating:
non-resource variables are not supported in the long term

 inside main printing x: Tensor("Placeholder:0", shape=(?, 227, 227, 3), dtype=float32)

 inside conv printing input : (?, 227, 227, 3)
 inside conv printing biases : (96,)
 inside conv printing k_h, k_w, c_o, s_h, s_w : 11 11 96 4 4
 inside conv printing conv : Tensor("Conv2D:0", shape=(?, 57, 57, 96), dtype=float32)
 inside conv printing conv_return : Tensor("Reshape:0", shape=(?, 57, 57, 96), dtype=float32)

 inside AlexNet printing conv1 : Tensor("Relu:0", shape=(?, 57, 57, 96), dtype=float32)
 inside AlexNet printing lrn1 : Tensor("LRN:0", shape=(?, 57, 57, 96), dtype=float32)
 inside AlexNet printing maxpool1 : Tensor("MaxPool:0", shape=(?, 28, 28, 96), dtype=float32)

 inside conv printing input : (?, 28, 28, 96)
 inside conv printing biases : (256,)
 inside conv printing k_h, k_w, c_o, s_h, s_w : 5 5 256 1 1
 inside conv printing input_groups : [<tf.Tensor 'split:0' shape=(?, 28, 28, 48) dtype

In [4]:
len(class_names)

1000

In [5]:
probs

<tf.Tensor 'Softmax:0' shape=(?, 1000) dtype=float32>